# Model input ERP format preparation 

In this notebook: 
- Necessary inputs
- Read all epochs
- Function to create dataframe with average mismatch response for all participants (needs to be transformed to function)
- Formatting dataframe as suitable model input

## Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from math import nan
from scipy import stats

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata

,eeg_file,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,101a,101,a,m,20,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,101a_epo.fif
1,101b,101,b,m,23,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,101b_epo.fif
2,102a,102,a,f,20,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,102a_epo.fif
3,102b,102,b,f,23,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,102b_epo.fif
4,103a,103,a,f,20,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,103a_epo.fif
...,...,...,...,...,...,...,...,...,...,...,...
173,194a,194,a,f,19,f,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,194a_epo.fif
174,194b,194,b,f,22,f,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,194b_epo.fif
175,195a,195,a,f,18,f,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,195a_epo.fif
176,211a,211,a,m,18,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,211a_epo.fif


## Read all epochs from files

The function below loads all filtered epochs from the dataframe which contains the metadata and the epochs filepaths + filenames. The epochs are arrays for each stimuli with a time interval of -0.3 to 0.7.

In [6]:
epochs = initialization_functions.read_filtered_data(metadata)

Checking out file: 101a_epo.fif
Checking out file: 102a_epo.fif
Checking out file: 103a_epo.fif
Checking out file: 104a_epo.fif
Checking out file: 105a_epo.fif
Checking out file: 106a_epo.fif
Checking out file: 107a_epo.fif
Checking out file: 109a_epo.fif
Checking out file: 110a_epo.fif
Checking out file: 111a_epo.fif
Checking out file: 112a_epo.fif
Checking out file: 114a_epo.fif
Checking out file: 115a_epo.fif
Checking out file: 116a_epo.fif
Checking out file: 117a_epo.fif
Checking out file: 118a_epo.fif
Checking out file: 119a_epo.fif
Checking out file: 123a_epo.fif
Checking out file: 124a_epo.fif
Checking out file: 125a_epo.fif
Checking out file: 126a_epo.fif
Checking out file: 127a_epo.fif
Checking out file: 128a_epo.fif
Checking out file: 129a_epo.fif
Checking out file: 130a_epo.fif
Checking out file: 131a_epo.fif
Checking out file: 133a_epo.fif
Checking out file: 135a_epo.fif
Checking out file: 136a_epo.fif
Checking out file: 137a_epo.fif
Checking out file: 138a_epo.fif
Checking

In [8]:
len(epochs)

90

## Create pandas dataframe with the average difference between standard and deviant responses

The function below needs `metadata`, the loaded `epochs` and the definition of the standard and deviant events as input. You should define your standard and deviant events as an array. In the function `input_mmr_prep` it's important to know that the assumption is made that the deviant follows after a standard event. Therefore the deviant belonging to the standard is the  standard event number + 1. Make sure your events are numbered like this, else the function won't calculate the mismatch response.  

In [9]:
def input_mmr_prep(metadata, epochs, standard_events): 
    # create dataframe with expected columns 
    df = pd.DataFrame(columns=["eeg_file",  "channel", "mean"]) # "paradigm",

    # loop over all participants
    for i in range(len(metadata['eeg_file'])):
        std_evoked = epochs[i][standard_events].average() 
        dev_evoked = epochs[i][deviant_events].average()

        # calculate the mismatch response between standard and deviant evoked
        evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])
        
        # get a list of all channels
        chnames_list = evoked_diff.info['ch_names']
        
        # compute for every channel the features of the mismatch line
        for channel in chnames_list: 
            chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
            roi_dict = dict(left_ROI=chnames) # combine_channels only takes a dictionary as input
            roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
            mmr = roi_evoked.to_data_frame()
            mmr_avg = mmr['left_ROI'].mean()
            mmr_std = mmr['left_ROI'].std()
            mmr_skew = mmr['left_ROI'].skew()
            mmr_var = mmr['left_ROI'].var()
            mmr_kurt = mmr['left_ROI'].kurtosis()
            
            df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew, 'kurt' : mmr_kurt, 'var' : mmr_var}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms 
    return df

In [10]:
# define the events for standard and deviant
standard_events = [2,5,8,11]
deviant_events = [3,6,9,12]

df = input_mmr_prep(metadata, epochs, standard_events, deviant_events)

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

...

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-9-1cc51afed815>:31: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x000001ECF9CA9C10>, axis=0)".
  roi_evoke

In [11]:
df

,eeg_file,channel,mean,kurt,skew,std,var
0,101a,Fp1,3.802558,0.648213,-1.088595,13.966382,195.059830
1,101a,AF3,-8.698850,0.420286,-0.941927,11.072648,122.603539
2,101a,F7,15.793048,-1.399260,-0.222532,14.460686,209.111439
3,101a,F3,-13.212232,0.864322,-1.011522,14.516818,210.738005
4,101a,FC1,-20.837596,-0.451340,0.154026,16.835498,283.433999
...,...,...,...,...,...,...,...
2875,212a,F8,-17.668922,1.515680,1.503259,21.854100,477.601680
2876,212a,AF4,-25.471698,1.538635,1.598874,23.931974,572.739389
2877,212a,Fp2,-23.625687,1.704133,1.658742,24.580506,604.201282
2878,212a,Fz,-31.196859,0.720195,1.312814,27.456152,753.840307


In [12]:
df = df.drop_duplicates(subset=['eeg_file','channel']) # ,'paradigm'

## Transpose dataframe into combination of paradigm and channel per participant

We now want a single row for every participant containing the paradigm and corresponding channels. The code below generates this dataframe. 

In [13]:
# transformation of the dataframe
df = df.pivot(index='eeg_file', columns=['channel']) # 'paradigm',
df

mean                                                         \
channel         AF3        AF4         C3         C4        CP1        CP2   
eeg_file                                                                     
101a      -8.698850 -23.709517 -14.116824 -19.310100  -2.458536 -26.754174   
102a     -12.651143 -17.591247 -14.895518 -10.553126  -3.815776  -4.456747   
103a      -4.291754  -7.721236 -25.840885 -21.765084 -33.959699 -34.063565   
104a     -23.586452 -27.014460 -14.197314 -12.797756 -19.694582 -19.994248   
105a      -3.473711  -2.229290   0.588515  -0.464928   7.717058  11.191306   
...             ...        ...        ...        ...        ...        ...   
216a     -41.444634 -25.633972 -32.105548 -50.839191 -46.723724 -29.315308   
217a       1.937432  14.313379  -0.034367  14.749652   2.544271  13.065405   
218a       4.080161   7.790090  -3.307886  -5.131242  -1.775464  -3.360262   
219a       0.307460  -3.515791  -5.737908  -7.957848  -4.395491  -4.698619   
220a      36.170632  40.753726  37.028210  44.805384  35.240712  37.322155   

                                                      ...          var  \
channel         CP5        CP6         Cz         F3  ...           Oz   
eeg_file                                              ...                
101a     -10.368958 -22.028517 -21.244704 -13.212232  ...   806.209372   
102a      -7.756276  -1.770537 -11.983060  -8.635793  ...  1099.091799   
103a     -32.614270 -16.372054 -33.627666  -6.652452  ...  1460.068447   
104a     -11.238579  -6.616907 -27.799114 -20.336733  ...    66.067646   
105a       2.232668   0.528495   9.460590   4.738113  ...   300.966973   
...             ...        ...        ...        ...  ...          ...   
216a      -4.623241 -19.818626 -37.566254 -33.336196  ...  1621.582168   
217a       5.469382  25.247935   6.492388  -0.678825  ...   476.255910   
218a      -3.148090  -4.764215   5.065358   0.372969  ...   156.728356   
219a     -26.413288 -28.337591  12.101671   3.309770  ...   487.859829   
220a      40.484925  52.160731  34.161744  32.995484  ...  1384.754707   

                                                                           \
channel            P3           P4           P7           P8          PO3   
eeg_file                                                                    
101a        76.921450   506.858822   136.716508   337.816997   384.993960   
102a       227.089867   532.484306  1213.374987   541.131597   602.439210   
103a       501.195654   778.926598   303.998905   496.661763   867.650545   
104a       144.205339   129.565302   183.609953   131.355955   111.983206   
105a       255.443200   226.412104   137.164369   404.895588   257.591693   
...               ...          ...          ...          ...          ...   
216a      1159.340951  2963.267685   231.381299   112.791440  1384.341960   
217a        56.052555   383.774160   171.277159   173.994679   213.585422   
218a       106.058416   163.213133   106.152805    94.708524    96.529695   
219a       575.594484   854.091453   540.599299   347.410313   565.697081   
220a      1352.114323  1154.257364   940.414723  4857.530928  1709.206439   

                                                              
channel           PO4           Pz           T7           T8  
eeg_file                                                      
101a       244.368925   200.547652   113.608744   355.856014  
102a       497.709783   576.354850   546.263357   320.299078  
103a       797.421991   567.904304   337.769169   302.277737  
104a       172.833291   138.677103   154.772343    95.081613  
105a       387.939507   203.877964   145.847537    77.076837  
...               ...          ...          ...          ...  
216a      1315.873799  1741.909484   182.765358  1221.561377  
217a       252.498077   217.125857   235.590776   436.839069  
218a       277.776943   236.351164    79.117091   117.409253  
219a      1417.416897   787.046765   256.082444   

In [14]:
df.columns = ['_'.join(str(s).strip() for s in col if s) for col in df.columns]
df

,mean_AF3,mean_AF4,mean_C3,mean_C4,mean_CP1,mean_CP2,mean_CP5,mean_CP6,mean_Cz,mean_F3,...,var_Oz,var_P3,var_P4,var_P7,var_P8,var_PO3,var_PO4,var_Pz,var_T7,var_T8
eeg_file,,,,,,,,,,,,,,,,,,,,,
101a,-8.698850,-23.709517,-14.116824,-19.310100,-2.458536,-26.754174,-10.368958,-22.028517,-21.244704,-13.212232,...,806.209372,76.921450,506.858822,136.716508,337.816997,384.993960,244.368925,200.547652,113.608744,355.856014
102a,-12.651143,-17.591247,-14.895518,-10.553126,-3.815776,-4.456747,-7.756276,-1.770537,-11.983060,-8.635793,...,1099.091799,227.089867,532.484306,1213.374987,541.131597,602.439210,497.709783,576.354850,546.263357,320.299078
103a,-4.291754,-7.721236,-25.840885,-21.765084,-33.959699,-34.063565,-32.614270,-16.372054,-33.627666,-6.652452,...,1460.068447,501.195654,778.926598,303.998905,496.661763,867.650545,797.421991,567.904304,337.769169,302.277737
104a,-23.586452,-27.014460,-14.197314,-12.797756,-19.694582,-19.994248,-11.238579,-6.616907,-27.799114,-20.336733,...,66.067646,144.205339,129.565302,183.609953,131.355955,111.983206,172.833291,138.677103,154.772343,95.081613
105a,-3.473711,-2.229290,0.588515,-0.464928,7.717058,11.191306,2.232668,0.528495,9.460590,4.738113,...,300.966973,255.443200,226.412104,137.164369,404.895588,257.591693,387.939507,203.877964,145.847537,77.076837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216a,-41.444634,-25.633972,-32.105548,-50.839191,-46.723724,-29.315308,-4.623241,-19.818626,-37.566254,-33.336196,...,1621.582168,1159.340951,2963.267685,231.381299,112.791440,1384.341960,1315.873799,1741.909484,182.765358,1221.561377
217a,1.937432,14.313379,-0.034367,14.749652,2.544271,13.065405,5.469382,25.247935,6.492388,-0.678825,...,476.255910,56.052555,383.774160,171.277159,173.994679,213.585422,252.498077,217.125857,235.590776,436.839069
218a,4.080161,7.790090,-3.307886,-5.131242,-1.775464,-3.360262,-3.148090,-4.764215,5.065358,0.372969,...,156.728356,106.058416,163.213133,106.152805,94.708524,96.529695,277.776943,236.351164,79.117091,117.409253


In [15]:
df.reset_index(inplace=True)
df

,eeg_file,mean_AF3,mean_AF4,mean_C3,mean_C4,mean_CP1,mean_CP2,mean_CP5,mean_CP6,mean_Cz,...,var_Oz,var_P3,var_P4,var_P7,var_P8,var_PO3,var_PO4,var_Pz,var_T7,var_T8
0,101a,-8.698850,-23.709517,-14.116824,-19.310100,-2.458536,-26.754174,-10.368958,-22.028517,-21.244704,...,806.209372,76.921450,506.858822,136.716508,337.816997,384.993960,244.368925,200.547652,113.608744,355.856014
1,102a,-12.651143,-17.591247,-14.895518,-10.553126,-3.815776,-4.456747,-7.756276,-1.770537,-11.983060,...,1099.091799,227.089867,532.484306,1213.374987,541.131597,602.439210,497.709783,576.354850,546.263357,320.299078
2,103a,-4.291754,-7.721236,-25.840885,-21.765084,-33.959699,-34.063565,-32.614270,-16.372054,-33.627666,...,1460.068447,501.195654,778.926598,303.998905,496.661763,867.650545,797.421991,567.904304,337.769169,302.277737
3,104a,-23.586452,-27.014460,-14.197314,-12.797756,-19.694582,-19.994248,-11.238579,-6.616907,-27.799114,...,66.067646,144.205339,129.565302,183.609953,131.355955,111.983206,172.833291,138.677103,154.772343,95.081613
4,105a,-3.473711,-2.229290,0.588515,-0.464928,7.717058,11.191306,2.232668,0.528495,9.460590,...,300.966973,255.443200,226.412104,137.164369,404.895588,257.591693,387.939507,203.877964,145.847537,77.076837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,216a,-41.444634,-25.633972,-32.105548,-50.839191,-46.723724,-29.315308,-4.623241,-19.818626,-37.566254,...,1621.582168,1159.340951,2963.267685,231.381299,112.791440,1384.341960,1315.873799,1741.909484,182.765358,1221.561377
85,217a,1.937432,14.313379,-0.034367,14.749652,2.544271,13.065405,5.469382,25.247935,6.492388,...,476.255910,56.052555,383.774160,171.277159,173.994679,213.585422,252.498077,217.125857,235.590776,436.839069
86,218a,4.080161,7.790090,-3.307886,-5.131242,-1.775464,-3.360262,-3.148090,-4.764215,5.065358,...,156.728356,106.058416,163.213133,106.152805,94.708524,96.529695,277.776943,236.351164,79.117091,117.409253
87,219a,0.307460,-3.515791,-5.737908,-7.957848,-4.395491,-4.698619,-26.413288,-28.337591,12.101671,...,487.859829,575.594484,854.091453,540.599299,347.410313,565.697081,1417.416897,787.046765,256.082444,476.147113


## Merge and safe dataframe

We still need to merge some of the metadata into the dataframe, so we have the information of the age, gender and label of the participant. 

In [16]:
df = pd.merge(df, metadata, on='eeg_file')

In [17]:
pd.set_option('display.max_columns', None)

In [18]:
df

,eeg_file,mean_AF3,mean_AF4,mean_C3,mean_C4,mean_CP1,mean_CP2,mean_CP5,mean_CP6,mean_Cz,mean_F3,mean_F4,mean_F7,mean_F8,mean_FC1,mean_FC2,mean_FC5,mean_FC6,mean_Fp1,mean_Fp2,mean_Fz,mean_O1,mean_O2,mean_Oz,mean_P3,mean_P4,mean_P7,mean_P8,mean_PO3,mean_PO4,mean_Pz,mean_T7,mean_T8,kurt_AF3,kurt_AF4,kurt_C3,kurt_C4,kurt_CP1,kurt_CP2,kurt_CP5,kurt_CP6,kurt_Cz,kurt_F3,kurt_F4,kurt_F7,kurt_F8,kurt_FC1,kurt_FC2,kurt_FC5,kurt_FC6,kurt_Fp1,kurt_Fp2,kurt_Fz,kurt_O1,kurt_O2,kurt_Oz,kurt_P3,kurt_P4,kurt_P7,kurt_P8,kurt_PO3,kurt_PO4,kurt_Pz,kurt_T7,kurt_T8,skew_AF3,skew_AF4,skew_C3,skew_C4,skew_CP1,skew_CP2,skew_CP5,skew_CP6,skew_Cz,skew_F3,skew_F4,skew_F7,skew_F8,skew_FC1,skew_FC2,skew_FC5,skew_FC6,skew_Fp1,skew_Fp2,skew_Fz,skew_O1,skew_O2,skew_Oz,skew_P3,skew_P4,skew_P7,skew_P8,skew_PO3,skew_PO4,skew_Pz,skew_T7,skew_T8,std_AF3,std_AF4,std_C3,std_C4,std_CP1,std_CP2,std_CP5,std_CP6,std_Cz,std_F3,std_F4,std_F7,std_F8,std_FC1,std_FC2,std_FC5,std_FC6,std_Fp1,std_Fp2,std_Fz,std_O1,std_O2,std_Oz,std_P3,std_P4,std_P7,std_P8,std_PO3,std_PO4,std_Pz,std_T7,std_T8,var_AF3,var_AF4,var_C3,var_C4,var_CP1,var_CP2,var_CP5,var_CP6,var_Cz,var_F3,var_F4,var_F7,var_F8,var_FC1,var_FC2,var_FC5,var_FC6,var_Fp1,var_Fp2,var_Fz,var_O1,var_O2,var_Oz,var_P3,var_P4,var_P7,var_P8,var_PO3,var_PO4,var_Pz,var_T7,var_T8,ParticipantID,test,sex,age_months,age_months_days,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,101a,-8.698850,-23.709517,-14.116824,-19.310100,-2.458536,-26.754174,-10.368958,-22.028517,-21.244704,-13.212232,-25.386179,15.793048,-17.056091,-20.837596,-36.228070,-1.111741,-25.762392,3.802558,-10.622806,-25.279985,33.946183,20.423181,32.803110,-6.460419,-26.216826,5.977925,-13.941393,19.215243,-9.120554,-8.675925,-0.157604,-16.656226,0.420286,-1.262284,-0.873065,-0.912018,0.440981,-0.970993,-0.303408,-1.191184,-0.593721,0.864322,-0.926158,-1.399260,-1.027573,-0.451340,-1.095847,2.358449,-1.035204,0.648213,0.033603,-0.901768,-0.972186,-0.640471,-0.927427,-1.005154,-1.061982,-0.925817,-1.019576,-0.980291,-0.678708,-0.426082,-0.397631,-0.794546,-0.941927,0.154711,0.201777,0.123152,-0.868958,0.276725,0.406823,0.221337,0.067211,-1.011522,0.169576,-0.222532,-0.273782,0.154026,0.332576,-1.377932,0.042146,-1.088595,-0.724912,0.202838,0.166924,-0.004982,0.104461,-0.172729,-0.016983,0.090384,-0.344653,0.400216,-0.647920,-0.294624,-0.467220,-0.326820,11.072648,18.338733,12.350545,17.929106,10.353579,21.050456,10.164145,22.404182,17.721315,14.516818,19.544216,14.460686,16.421673,16.835498,27.162756,9.136868,21.438485,13.966382,11.184934,20.252166,29.637996,19.308707,28.393826,8.770487,22.513525,11.692583,18.379799,19.621263,15.632304,14.161485,10.658740,18.864146,122.603539,336.309123,152.535956,321.452841,107.196589,443.121693,103.309848,501.947377,314.045001,210.738005,381.976394,209.111439,269.671335,283.433999,737.815324,83.482352,459.608620,195.059830,125.102742,410.150240,878.410779,372.826160,806.209372,76.921450,506.858822,136.716508,337.816997,384.993960,244.368925,200.547652,113.608744,355.856014,101,a,m,20,20;22,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ev...,101a_epo.fif
1,102a,-12.651143,-17.591247,-14.895518,-10.553126,-3.815776,-4.456747,-7.756276,-1.770537,-11.983060,-8.635793,-11.386706,-2.231622,-4.575328,-11.245031,-13.980653,-5.532752,-4.318029,-14.203784,-17.443230,-16.426453,10.650066,24.723190,21.779760,4.321163,-2.825715,21.530274,-10.376464,19.770339,1.576042,10.915654,0.062860,-3.059482,-0.353253,0.151132,-0.640433,-0.282760,-0.216171,-0.596513,-0.520595,-0.179884,-0.613825,-0.022511,-0.692120,0.877791,-0.329822,-0.586081,-0.708147,-0.049100,-0.699985,-0.217358,0.181259,-0.417417,-0.756724,-0.326616,0.016184,-0.435956,-0.200733,-0.566021,-0.931568,-0.934610,-0.827786,-0.967326,-0.380317,-0.328436,0.191341,0.563803,-0.244465,0.775263,-0.239471,0.177145,-0.282280,0.357027,0.081801,0.575924,0.2301

Drop some unnecessary columns. 

In [19]:
df = df.drop(['eeg_file','age_months_days',
       'dyslexic_parent', 'path_eeg','path_epoch',
       'epoch_file', 'path_eventmarkers'], axis =1)

In [25]:
df['sex'] = np.where(
    (df['sex']=='m'), 1,0)

df['Group_AccToParents'] = np.where(
    (df['Group_AccToParents']=='At risk'), 1,0)

In [26]:
first = df.pop('Group_AccToParents')
df.insert(0, 'Group_AccToParents', first)

To remove some outliers on the data, the z score is calculated. 

In [ ]:
#df = df[(np.abs(stats.zscore(df)) < 5).all(axis=1)] # z score 

In [28]:
df.to_csv('df_avg_mmr.csv', index=False) # safe dataframe

## PCA analysis on feature reduction 

In [ ]:
X = df.drop('Group_AccToParents',1)
y = df['Group_AccToParents']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
explained_variance = pca.explained_variance_ratio_

In [ ]:
explained_variance

In [ ]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

In [ ]:
X.shape